In [1]:
import os,shutil
def mkdir(path):
    if not os.path.exists(path):
        os.makedirs(path)

In [1]:
# merge and rename
import shutil
import os
path = "/msl/home/kelly08385/Pictures/Fast_defect/8800_0307/data"
despath = "/msl/home/kelly08385/Pictures/Fast_defect/8800_0307/JPEGImages"
for root,dirs,files in os.walk(path):
    for file in files:
        #dir = root.split('/')[-1]
        shutil.copy(os.path.join(root,file),os.path.join(os.path.join(despath,file)))
        #os.rename(os.path.join(root,file),os.path.join(despath, dir, file[:-4]+'.jpg'))

## use original image and crop image to find xml 

In [29]:
import xml.etree.cElementTree as ET
import cv2
import os
import numpy as np
import shutil

crop_path = "/msl/home/kelly08385/Pictures/Fast_defect/8851/crop"
des_crop_path = "/msl/home/kelly08385/Pictures/Fast_defect/8851/train_crop"
ori_path = "/msl/home/kelly08385/Pictures/Fast_defect/8851/ori"
des_path = "/msl/home/kelly08385/Pictures/Fast_defect/8851/data"
des_path2 = "/msl/home/kelly08385/Pictures/Fast_defect/8851/JPEGImages"
xml_path = "/msl/home/kelly08385/Pictures/Fast_defect/8851/Annotations"

#quiz_path = "/home/kelly/Pictures/defect/0312_work/9300/quiz"
        

In [ ]:
for root,dirs,files in os.walk(crop_path):
    dir = root.split("/")[-1]
    for file in files:
        print()
        mkdir(os.path.join(des_path,dir))
        mkdir(os.path.join(des_crop_path,dir))
        if os.path.exists(os.path.join(ori_path,dir,file[:-4]+".jpg")):

            try:
                cropImg = cv2.imread(os.path.join(root,file))
                cropImg = cv2.cvtColor(cropImg, cv2.COLOR_BGR2GRAY)
                oriImg = cv2.imread(os.path.join(ori_path,dir,file[:-4]+".jpg"))
                oriImg = cv2.cvtColor(oriImg, cv2.COLOR_BGR2GRAY)
                
                result = cv2.matchTemplate(oriImg, cropImg, cv2.TM_CCOEFF_NORMED) 
                x_and_y = np.unravel_index(result.argmax(),result.shape)
                print(x_and_y)

                tree = ET.parse('/msl/home/kelly08385/Pictures/trained_defect/total/Annotations/2300AAHL5AHYC14C100007-03.xml')
                tree_root = tree.getroot()
                print(tree_root)
                change_dict = {"filename":file,"width":oriImg.shape[1],"height":oriImg.shape[0],"xmin":x_and_y[1],"ymin":x_and_y[0],\
                               "xmax":x_and_y[1]+cropImg.shape[1],"ymax":x_and_y[0]+cropImg.shape[0],"name":root.split("/")[-1]}
                for key in change_dict.keys():
                    for e in tree_root.iter(key):
                        e.text = str(change_dict[key])
                shutil.copy(os.path.join(ori_path,dir,file),os.path.join(des_path2,file))
                shutil.move(os.path.join(ori_path,dir,file),os.path.join(des_path,dir,file))
                shutil.move(os.path.join(root,file),os.path.join(des_crop_path,dir,file))
                tree.write(os.path.join(xml_path,file[:-4]+".xml"))
            except Exception as e:
                print(str(e))

In [ ]:
# use xml to catch picture
ori_path = "/home/kelly/Pictures/defect/0322_work/4800/no_rec"
crop_path = "/home/kelly/Pictures/defect/0322_work/4800/rec"
xml_path = "/home/kelly/Pictures/fastrcnn/4800_0323/result/Annotations"
des_path = "/home/kelly/Pictures/fastrcnn/4800_0323/data"

for root,dirs,files in os.walk(crop_path):
    for file in files:
        dir = root.split('/')[-1]
        print(os.path.join(ori_path,dir,file[:-4]+".jpg"))
        print(os.path.join(xml_path,file[:-4]+".xml"))
        desPath = os.path.join(des_path,dir)
        print(desPath)
#         if os.path.exists(os.path.join(ori_path,dir,file[:-4]+".jpg")) and\
#                           os.path.exists(os.path.join(xml_path,file[:-4]+".xml")):
#             mkdir(desPath)
#             shutil.move(os.path.join(ori_path,dir,file[:-4]+".jpg"),os.path.join(des_path,dir,file[:-4]+".jpg"))

-----------------------------------------------------------------------
## 由紅框產生xml

In [2]:
# find red rec
from scipy.misc import imread, imsave
import numpy as np
from skimage.segmentation import clear_border
from skimage.measure import regionprops, label
from skimage.morphology import erosion,square


def size(a, b=0, nargout=1):
    s = np.asarray(a).shape
    if s is ():
        return 1 if b else (1,)*nargout
    # a is not a scalar
    try:
        if b:
            return s[b-1]
        elif nargout > 1:
            return s
    except Exception:
        return 1
    return  1
 
def im2double(im):
    info = np.iinfo(im.dtype)
    return im.astype(np.float) / info.max

def rgb2gray(rgb):
    return np.dot(rgb[...,:3], [0.299, 0.587, 0.114])

def double(img):
    return img.astype(np.float)

def getImgInRedRect(imgPath):
        ret = ''
        path_arr = imgPath.split(os.path.sep)
        rawImg = imread(imgPath)
        if size(rawImg,3) < 3:
            return ret
 
        tmpImg = np.copy(rawImg)
        tmpImg = im2double(tmpImg)
        tmpImg[:50,:50,:] = 0
        tmpImg[-50:,:50,:] = 0
        tmpImg[-50:,-50:,:] = 0
        tmpImg[:50,-50:,:] = 0
        tmpImg[:,:,0] = tmpImg[:,:,0] > 0.8
        tmpImg[:,:,1] = tmpImg[:,:,1] < 0.2
        tmpImg[:,:,2] = tmpImg[:,:,2] < 0.2
        tmpImg = rgb2gray(tmpImg)
        tmpImg = double(tmpImg > 0.8)
        bw = tmpImg > 0.8
        bw = bw.astype(int)
        #cleared = clear_border(bw)
        stats = regionprops(label(bw))
        
        flag = False
        #print (imgPath)
        #print(output, output2, output3)
        maxArea = 500
        for currentStats in stats:
            if currentStats.area > maxArea:
                minr, minc, maxr, maxc = currentStats.bbox
                x = int(max(round(minr + 10), 1))
                y = int(min(round(maxc - 10), size(rawImg,2)))
                maxArea = currentStats.area
                crop = None
                if int(min(round(maxr - 10), size(rawImg,1))) > x and y > int(max(round(minc + 10) , 1)):
                    crop = rawImg[x:int(min(round(maxr - 10), size(rawImg,1))), int(max(round(minc + 10) , 1)):y,:]
                flag = True
#             returnX = x
#             returnY = int(max(round(minc + 10) , 1))

        if flag:
            return crop
        else:
            return None
                #print ("red rec:", output)
 


In [5]:
import cv2,os,shutil
import xml.etree.cElementTree as ET

# create xml

def create_xml(ori_path,root,file,xml_path,data_path,data_path2,des_rec):
    
    imgPath = os.path.join(root,file)
    ori_path = os.path.join(ori_path,root.split('/')[-1])
    # 原圖不存在
    if not os.path.exists(os.path.join(ori_path,file[:-4]+".jpg")):
        return
    oriImg = cv2.imread(os.path.join(ori_path,file[:-4]+".jpg"))
    oriImg = cv2.cvtColor(oriImg, cv2.COLOR_BGR2GRAY)
        
    #xml_path = os.path.join(xml_path,root.split('/')[-1])
    data_path = os.path.join(data_path,root.split('/')[-1])
    des_rec_path = os.path.join(des_rec,root.split('/')[-1])
    mkdir(des_rec_path)
    mkdir(data_path)

    cropImg = getImgInRedRect(imgPath)
    
    if not cropImg is None :
        cropImg = cv2.cvtColor(cropImg, cv2.COLOR_BGR2GRAY)
        oriImg.astype(np.uint8)
        result = cv2.matchTemplate(oriImg, cropImg, cv2.TM_CCOEFF_NORMED) 
        x_and_y = np.unravel_index(result.argmax(),result.shape)
        print(x_and_y)

        tree = ET.parse('/msl/home/kelly08385/Pictures/trained_defect/total_0630/Annotations/2300AAHL5AHYC14C100007-03.xml')
        tree_root = tree.getroot()
        change_dict = {"filename":file[:-4]+".xml","width":oriImg.shape[1],"height":oriImg.shape[0],"xmin":x_and_y[1],"ymin":x_and_y[0],\
                                       "xmax":x_and_y[1]+cropImg.shape[1],"ymax":x_and_y[0]+cropImg.shape[0],"name":root.split('/')[-1]}

        for key in change_dict.keys():
            for e in tree_root.iter(key):
                e.text = str(change_dict[key])
                tree.write(os.path.join(xml_path,file[:-4]+".xml"))
        shutil.copy(os.path.join(ori_path,file),os.path.join(data_path2,file))
        shutil.move(os.path.join(ori_path,file),os.path.join(data_path,file))
        shutil.move(imgPath,os.path.join(des_rec_path,file))


In [8]:
# use demooutput to create xml
# use rec to find no rec

ori_path = "/msl/home/kelly08385/Pictures/Fast_defect/2851/ori"
rec_path = "/msl/home/kelly08385/Pictures/Fast_defect/2851/rec"
xml_path = "/msl/home/kelly08385/Pictures/Fast_defect/2851/Annotations"
data_path = "/msl/home/kelly08385/Pictures/Fast_defect/2851/data"
data_path2 = "/msl/home/kelly08385/Pictures/Fast_defect/2851/JPEGImages"
des_rec = "/msl/home/kelly08385/Pictures/Fast_defect/2851/train_rec"

for root, dirs, files in os.walk(rec_path):
    for file in files:
        print(file)
        create_xml(ori_path,root,file,xml_path,data_path,data_path2,des_rec)


2856TL83J10AP134100001-01.jpg


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:36: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.


(718, 606)
2856TL8264FAC1AH100001-01.jpg
(518, 727)
2856TL82P1VAE107100001-01.jpg
(444, 702)
2856AAHL6ARXQ127100001-03.jpg
(355, 562)
2856TL7CM2GAV10G100001-01.jpg
(383, 625)
2856TL83H3AAK15L100001-01.jpg
(356, 702)
2856TL83E2PBA123100001-01.jpg
(413, 774)
2856TL82H2EAZ179100001-01.jpg
(398, 583)
2856TL83D1DAN1BM100001-01.jpg
(446, 632)
2856TL82H2EAW149100001-01.jpg
(481, 546)
2856TL82G1PAY1N7100001-01.jpg
(663, 734)
2856TL82Q36AT183100001-01.jpg
(406, 625)
2856TL83H3KAH176100001-01.jpg
(430, 732)
2856TL7AD3RAX1D8100001-01.jpg
(643, 810)
2856TL83J10BD163100001-01.jpg
(352, 532)
2856AAHL6AQXW116100001-03.jpg
(434, 646)
2856TL7BB1GAT1M1100001-01.jpg
(526, 643)
2856TL7CK3JAW122100002-01.jpg
(316, 548)
2856TL7AM1NAV105100001-01.jpg
(294, 387)
2856TL8254ABA141100001-01.jpg
(242, 567)
2856TL7AD05AF10L100001-01.jpg
(486, 592)
2856TL7BJ49BA1B6100001-01.jpg
(479, 615)
2856TL82F3AAR168100001-01.jpg
(359, 660)
2856TL83K2SAG101100005-01.jpg
(222, 657)
2856TL8254AAC10J100001-01.jpg
(430, 570)
2856T

(69, 408)
2856AAHL6AQXE1F4100002-03.jpg
(210, 562)
2856AAHL6ACYA1B6100001-01.jpg
(422, 472)
2856TL7CN3MAB168100001-01.jpg
(320, 410)
2856TL7CK0DBC151100001-05.jpg
(10, 648)
2856TL7CM3AAS1D1100001-01.jpg
(10, 494)
2856TL7BJ49AY179100001-01.jpg
2856TL7C23XAK143100002-01.jpg
(636, 521)
2856TL7BJ3AAD181100001-01.jpg
2856AAHL3AXXA129100003-01.jpg
(109, 464)
2856TL7BC1ZAA114100001-01.jpg
(10, 10)
2856TL7C23XAN113100001-01.jpg
(776, 393)
2856TL7C21LAC11D100001-01.jpg
(11, 303)
2856AAHL3AXXQ1B2100001-01.jpg
(842, 480)
2856TL7C23XAC1A2100003-03.jpg
2856TL7C22EAM1E1100002-01.jpg
(18, 60)
2856TL7C23XAE135100002-01.jpg
(804, 540)
2856AAHL6ARXQ1S6100001-03.jpg
(553, 653)
2856TL7C23XAH163100002-03.jpg
2856TL7C23MBD10C100002-01.jpg
2856TL7B31RAF1D7100004-01.jpg
2856TL7BC0UAM1A2100002-01.jpg
(336, 562)
2856TL7B62BAB178100002-01.jpg
2856TL7CN1KAU104100001-01.jpg
(806, 610)
2856TL7BJ3AAA1M3100001-01.jpg
(83, 473)
2856TL7BB1GAG1P6100001-01.jpg
2856AAHL3AXXA1G7100001-01.jpg
(903, 605)
2856TL7C204AN1011000

(806, 736)
2856TL7BH3NAX151100001-01.jpg
2856TL7AB0XAZ1C1100001-01.jpg
(10, 376)
2856TL7BB4GAB1Q2100001-01.jpg
2856TL7A807AL103100001-01.jpg
(338, 369)
2856TL7BJ3AAC1D3100001-01.jpg
(268, 342)
2856TL7C22EAL176100002-01.jpg
(264, 10)
2856TL7AH0HAU175100001-01.jpg
(318, 10)
2856TL7CN3MAC198100001-01.jpg
(708, 492)
2856TL7AH0HAW187100001-01.jpg
2856TL7CM2JBD1A6100001-01.jpg
(305, 508)
2856TL7CK07AP136100001-01.jpg
(807, 674)
2856TL7CN04AC1K3100001-01.jpg
(319, 590)
2856TL7B60WBC1A8100002-01.jpg
(423, 604)
2856TL7B648BD101100001-01.jpg
(165, 464)
2856TL7C202BB1N3100002-01.jpg
(284, 540)
2856TL7B528AT183100001-03.jpg
(364, 711)
2856TL7BB1GAB135100002-03.jpg
(368, 583)
2856TL7B60WBC1A1100003-01.jpg
(586, 618)
2856TL7C21LAH111100001-01.jpg
(732, 662)
2856TL7C202AX1F8100001-01.jpg
(327, 422)
2856TL7B648AZ1A2100001-01.jpg
(303, 543)
2856TL7CN1KAZ168100001-01.jpg
2856AAHL6ACYC1KA100001-01.jpg
2856TL7B62BAD181100002-01.jpg
(354, 570)
2856TL7CK3JAW179100002-01.jpg
(426, 513)
2856TL7C202AW19A100001

...

(383, 622)
2856TL7C21LAL146100002-01.jpg
(400, 698)
2856TL7C21LAL154100001-01.jpg
(338, 567)
2856TL7CK0CBD18F100001-01.jpg
(238, 604)
2856TL7CM3AAA1E3100002-01.jpg
(326, 532)
2856TL7CM3CAD1C2100001-01.jpg
(500, 629)
2856TL7CM3CAH1RA100001-01.jpg
(416, 636)
2856TL7CM3AAB1JA100001-01.jpg
(404, 540)
2856TL7CM2JBC1DK100001-01.jpg
(352, 664)
2856TL7CM3CAP1G1100001-01.jpg
(539, 678)
2856TL7CM3CAM1F9100001-01.jpg
(442, 570)
2856AAHL3AXXE1RB100001-01.jpg
(319, 560)
2856AAHL3AXXD103100001-01.jpg
(270, 476)
2856TL7CM3CAQ17A100001-01.jpg
(427, 676)
2856TL7CN05AC12A100005-01.jpg
(410, 674)
2856TL7CN05AC1J7100001-01.jpg
(612, 846)
2856TL7B42AAV10C100001-01.jpg
(315, 394)
2856AAHL3AXXB1E1100001-01.jpg
(127, 562)
2856TL7C21LAV1B9100002-01.jpg
(277, 436)
2856TL7B306AC19J100001-01.jpg
(346, 587)
2856TL7CN3MAB1C5100001-01.jpg
(427, 712)
2856TL7CN03AC18A100001-01.jpg
(437, 627)
2856TL7CM3AAF1N7100001-01.jpg
(242, 714)
2856TL7C21LAW19C100004-01.jpg
(345, 541)
2856TL7CM3AAB1K4100001-01.jpg
(288, 668)
2856T

(266, 513)
2856AAHL3AXXD1T1100001-01.jpg
(326, 536)
2856AAHL3AXXE1A5100001-01.jpg
(332, 348)
2856TL7C21LAA16H100001-01.jpg
(312, 590)
2856AAHL3AXXB1SA100001-01.jpg
(277, 660)
2856TL7CM3CAQ124100001-01.jpg
(444, 732)
2856AAHL3AXXD1A7100002-01.jpg
(319, 530)
2856TL7CK07AP196100003-01.jpg
(322, 643)
2856TL7CX39AW135100001-01.jpg
(443, 732)
2856AAHL3AXXA187100002-01.jpg
(538, 434)
2856TL7CX39AV11A100001-01.jpg
(340, 572)
2856TL7CM3CAG1BA100001-01.jpg
(502, 748)
2856TL7C21LAJ136100001-01.jpg
(657, 655)
2856TL7C21LAX151100001-01.jpg
(228, 555)
2856AAHL3AXXC134100001-01.jpg
(266, 627)
2856AAHL3AXXB1NC100001-01.jpg
(326, 594)
2856AAHL3AEXG1R2100013-01.jpg
(343, 242)
2856AAHL3AMXS10B100001-01.jpg
(378, 458)
2856TL7C21LAT10L100003-01.jpg
(415, 622)
2856TL7CM3CAY167100001-01.jpg
(408, 664)
2856TL7CK24AQ193100001-01.jpg
(420, 511)
2856AAHL3AXXA10C100002-01.jpg
(371, 497)
2856AAHL3AMXX146100001-01.jpg
(304, 683)
2856AAHL3AXXE1C9100001-01.jpg
(314, 469)
2856TL7C21LAU1BL100001-01.jpg
(399, 611)
2856T

(238, 520)
2856TL7AD06AM1CD100001-01.jpg
(347, 543)
2856AAHL5AEXC1NA100001-01.jpg
(231, 469)
2856TL7B306AC19C100002-01.jpg
(326, 446)
2856TL7B52EAE194100001-01.jpg
(378, 602)
2856AAHL4BFXA101100004-01.jpg
2856TL7CN1RAA1CG100001-01.jpg
(437, 776)
2856TL7AC2JAA14H100001-01.jpg
(439, 584)
2856TL7C23ZAF1GC100002-01.jpg
(305, 504)
2856TL7CN1PAH112100002-01.jpg
(364, 583)
2856TL7AD2SAY1E7100001-01.jpg
(480, 460)
2856TL7A807AX134100001-01.jpg
(238, 406)
2856AAHL3AXXW155100002-01.jpg
2856AAHL3AXXG1L1100001-01.jpg
2856TL7AD05AD1AC100001-01.jpg
(450, 629)
2856TL7CN1KAS196100001-01.jpg
(343, 600)
2856TL7B52EAX1L4100002-01.jpg
(471, 622)
2856TL7C21LAN1A7100001-01.jpg
2856TL7AD2SAQ1A6100001-01.jpg
2856TL7C23ZAG131100001-01.jpg
(842, 569)
2856TL7BB1GAE1H5100001-05.jpg
(423, 522)
2856AAHL5AEXZ1T2100002-01.jpg
2856TL7A82HAA135100008-01.jpg
(340, 106)
2856TL7C23SAB189100001-01.jpg
2856TL7CN1KAU12C100001-01.jpg
(195, 472)
2856AAHL5AEXZ165100002-01.jpg
(372, 501)
2856TL7AD06AN16L100001-01.jpg
(155, 428)


(139, 459)
2856TL7AD02AZ1B2100001-01.jpg
(554, 525)
2856TL7B53JAZ1B7100001-03.jpg
(486, 636)
2856AAHL3AXXG1KA100001-01.jpg
(190, 380)
2856TL7B53JAZ1B7100001-01.jpg
(316, 788)
2856TL7CN1KAW16F100001-01.jpg
(200, 676)
2856TL7B62BAA1GA100001-01.jpg
2856TL7C23MAA1TC100002-01.jpg
(357, 462)
2856AAHL3AXXG1PC100001-01.jpg
(256, 506)
2856TL7AL23AE1D2100002-01.jpg
(567, 578)
2856TL7C23ZAH1B7100001-01.jpg
(350, 583)
2856TL7CK07AY14L100001-01.jpg
(294, 483)
2856TL7CK0DAA12D100001-01.jpg
(338, 429)
2856TL7B501AH119100001-01.jpg
(394, 790)
2856TL7B306AC1A2100002-01.jpg
(362, 474)
2856TL7AW1VAE1Q3100001-01.jpg
(296, 436)
2856TL7CK0DAZ172100001-01.jpg
(497, 584)
2856TL7AD06AW1B7100001-01.jpg
(383, 520)
2856TL7C21LAK11C100004-01.jpg
(362, 620)
2856TL7B52EAC132100001-01.jpg
(279, 597)
2856TL7B63QAN126100001-01.jpg
(483, 576)
2856AAHL4BFXA1JC100001-01.jpg
(330, 443)
2856TL7CL03AC1BA100002-01.jpg
(538, 634)
2856TL7A807AX127100001-01.jpg
(332, 671)
2856TL7C23ZAF132100001-01.jpg
(438, 664)
2856TL7B42JAZ156

(280, 462)
2856TL7C21LAH132100001-01.jpg
(736, 598)
2856AAHL4ALXA1T5100001-01.jpg
(402, 460)
2856TL7C23ZAH1C7100001-01.jpg
(327, 594)
2856TL7BC12AD1F4100001-01.jpg
(371, 574)
2856TL7B52EAT172100001-01.jpg
(420, 676)
2856TL7AH0VAS10K100001-01.jpg
(446, 592)
2856TL7B429AX169100001-01.jpg
(326, 385)
2856TL7AD05AE10L100001-01.jpg
(492, 588)
2856TL7AD06AV103100001-01.jpg
(123, 480)
2856AAHL6ARXA1D6100001-03.jpg
2856TL7C23ZAM1H2100001-01.jpg
(240, 515)
2856TL7CM2LAK1C1100001-01.jpg
(558, 534)
2856AAHL3AXXX1C6100001-01.jpg
(670, 709)
2856TL7AH2BAC10J100001-01.jpg
(284, 499)
2856TL7AH2BAR12C100002-01.jpg
2856TL7C23ZAG137100003-01.jpg
(330, 406)
2856AAHL4AJXK114100001-01.jpg
(746, 492)
2856TL7B52EAN1L9100001-01.jpg
2856TL7AH28AC1A7100001-01.jpg
(330, 474)
2856TL7C21LAB108100001-01.jpg
2856TL7B63QBA1M5100001-01.jpg
(53, 643)
2856TL7BB2UAC1Q1100001-01.jpg
(221, 434)
2856TL7B52EAB126100002-01.jpg
(406, 529)
2856TL7AD06AM139100003-01.jpg
(539, 748)
2856TL7AH0VAC13C100001-01.jpg
(250, 459)
2856TL7B5

In [ ]:
def getImgInBlueRect(imgPath):
        ret = ''
        path_arr = imgPath.split(os.path.sep)
        rawImg = imread(imgPath)
        if size(rawImg,3) < 3:
            return ret
#         output = os.path.abspath(os.path.join(targetPath,'redRec'))
#         for index, path_ in enumerate(path_arr):
#             if path_ not in sourcePath.split(os.path.sep):
#                 output = os.path.join(output, path_)
 
        tmpImg = np.copy(rawImg)
        tmpImg = im2double(tmpImg)
        tmpImg[:,:,0] = tmpImg[:,:,0] < 0.2
        tmpImg[:,:,1] = tmpImg[:,:,1] < 0.2
        tmpImg[:,:,2] = tmpImg[:,:,2] > 0.8
        tmpImg = rgb2gray(tmpImg)
        tmpImg = double(tmpImg > 0.9)
        bw = tmpImg > 0.9
        bw = bw.astype(int)
        cleared = clear_border(bw)
        stats = regionprops(label(cleared))
        flag = False
        #print (imgPath)
        #print(output, output2, output3)
        for currentStats in stats:
            minr, minc, maxr, maxc = currentStats.bbox
            if currentStats.area > 1000 and (maxr-minr) > 25 and (maxc-minc) > 25: # red rectangle
                x = int(max(round(minr + 10), 1))
                y = int(min(round(maxc - 10), size(rawImg,2)))
                crop = rawImg[x:int(min(round(maxr - 10), size(rawImg,1))), int(max(round(minc + 10) , 1)):y,:]
                flag = True
                returnX = x
                returnY = int(max(round(minc + 10) , 1))
                #imsave(output, crop)
        if flag:
            return crop
        else:
            return None
                #print ("red rec:", output)

---------------------------------------------------

In [195]:
imagePath = "/mnt/defect39/815_Defect/紅框分類/20180327_done/紅框/"
outputPath = "/mnt/defect39/815_Defect/紅框分類/20180327_done/紅框/crop"
# imagePath = "/msl/home/kelly08385/Downloads/onfmout"
# outputPath = "/msl/home/kelly08385/Downloads/onfmout/crop"
for root,dirs,files in os.walk(imagePath):
    for file in files:
        # print(root,file)
        cropImg = getImgInRedRect(os.path.join(root,file))
        if not cropImg is None: 
            _outputPath = os.path.join(*root.split('/')[-3:])
            _outputPath = outputPath + '/' +_outputPath
            print(_outputPath,file)
            mkdir(_outputPath)
            imsave(os.path.join(_outputPath,file), cropImg)
            print(file)
            

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:36: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.


currentstats 347 503 678 757
maxArea 4573
/mnt/defect39/815_Defect/紅框分類/20180327_done/紅框/crop/C010/TL50B1AA/G 4800TL78P0YAM1HC100001-01.JPG
4800TL78P0YAM1HC100001-01.JPG
currentstats 309 629 608 872
maxArea 4232
/mnt/defect39/815_Defect/紅框分類/20180327_done/紅框/crop/C010/TL50B1AA/G 4800TL78P0YAN125100001-01.JPG
4800TL78P0YAN125100001-01.JPG


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:14: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.
  


currentstats 291 636 582 873
maxArea 4087
/mnt/defect39/815_Defect/紅框分類/20180327_done/紅框/crop/C010/TL50B1AA/G 4800TL78P0YAS1FB100001-01.JPG
4800TL78P0YAS1FB100001-01.JPG
currentstats 406 722 642 978
maxArea 3517
/mnt/defect39/815_Defect/紅框分類/20180327_done/紅框/crop/C010/TL50B1AA/G 4800TL78P0YAS1K1100001-01.JPG
4800TL78P0YAS1K1100001-01.JPG
currentstats 350 538 530 834
maxArea 3530
/mnt/defect39/815_Defect/紅框分類/20180327_done/紅框/crop/C010/TL50B1AA/G 4800TL78P0YAW107100001-01.JPG
4800TL78P0YAW107100001-01.JPG
currentstats 417 650 581 832
maxArea 2675
/mnt/defect39/815_Defect/紅框分類/20180327_done/紅框/crop/C010/TL50B1AA/G 4800TL78P0YAZ1F1100001-01.JPG
4800TL78P0YAZ1F1100001-01.JPG
currentstats 406 417 640 722
maxArea 4118
/mnt/defect39/815_Defect/紅框分類/20180327_done/紅框/crop/C010/TL50B1AA/G 4800TL78P0YBD1F1100001-01.JPG
4800TL78P0YBD1F1100001-01.JPG
currentstats 288 600 440 812
maxArea 2567
/mnt/defect39/815_Defect/紅框分類/20180327_done/紅框/crop/C010/TL50B1AA/G 4800TL78P0YBD1N7100001-01.JPG
4800TL78P0

currentstats 398 701 514 827
maxArea 1729
/mnt/defect39/815_Defect/紅框分類/20180327_done/紅框/crop/LS_R/TL50B1AA/G 4800TL78P1RAG1T3100001-01.JPG
4800TL78P1RAG1T3100001-01.JPG
currentstats 326 712 427 829
maxArea 1646
/mnt/defect39/815_Defect/紅框分類/20180327_done/紅框/crop/LS_R/TL50B1AA/G 4800TL78P1RAH13B100001-01.JPG
4800TL78P1RAH13B100001-01.JPG
currentstats 54 534 594 871
maxArea 6905
/mnt/defect39/815_Defect/紅框分類/20180327_done/紅框/crop/LS_R/TL50B1AA/G 4800TL78P1RBB114100001-01.JPG
4800TL78P1RBB114100001-01.JPG
currentstats 382 920 492 1050
maxArea 1813
/mnt/defect39/815_Defect/紅框分類/20180327_done/紅框/crop/LS_R/TL50B1AA/G 4800TL78P1SAG131100001-01.JPG
4800TL78P1SAG131100001-01.JPG
currentstats 426 838 544 948
maxArea 1754
/mnt/defect39/815_Defect/紅框分類/20180327_done/紅框/crop/LS_R/TL50B1AA/G 4800TL78P1SAG1H8100001-01.JPG
4800TL78P1SAG1H8100001-01.JPG
currentstats 363 724 551 936
maxArea 3120
/mnt/defect39/815_Defect/紅框分類/20180327_done/紅框/crop/LS_R/TL50B1AA/G 4800TL78P1SAJ1G6100001-01.JPG
4800TL78P1

...

currentstats 174 505 373 800
maxArea 3693
/mnt/defect39/815_Defect/紅框分類/20180327_done/紅框/crop/LS_R/TL57B1AA/G 4800TL7AR0DAF1B7100001-01.JPG
4800TL7AR0DAF1B7100001-01.JPG
currentstats 200 443 387 841
maxArea 4414
/mnt/defect39/815_Defect/紅框分類/20180327_done/紅框/crop/LS_R/TL57B1AA/G 4800TL7AR0DAF1K5100001-01.JPG
4800TL7AR0DAF1K5100001-01.JPG
currentstats 384 522 577 728
maxArea 3114
/mnt/defect39/815_Defect/紅框分類/20180327_done/紅框/crop/LS_R/TL57B1AA/G 4800TL7AR0DAF1N7100001-01.JPG
4800TL7AR0DAF1N7100001-01.JPG
currentstats 354 624 502 815
maxArea 2639
/mnt/defect39/815_Defect/紅框分類/20180327_done/紅框/crop/LS_R/TL57B1AA/G 4800TL7AR0DAF1P3100001-01.JPG
4800TL7AR0DAF1P3100001-01.JPG
currentstats 84 473 310 736
maxArea 3751
/mnt/defect39/815_Defect/紅框分類/20180327_done/紅框/crop/LS_R/TL57B1AA/G 4800TL7AR0DAG129100001-01.JPG
4800TL7AR0DAG129100001-01.JPG
currentstats 251 468 506 784
maxArea 4491
/mnt/defect39/815_Defect/紅框分類/20180327_done/紅框/crop/LS_R/TL57B1AA/G 4800TL7AR0DAG199100001-01.JPG
4800TL7AR0D

currentstats 326 391 496 564
maxArea 2643
/mnt/defect39/815_Defect/紅框分類/20180327_done/紅框/crop/LS_R/TL57B1AA/G 4800TL7AR0DAX114100001-01.JPG
4800TL7AR0DAX114100001-01.JPG
currentstats 262 633 510 922
maxArea 4197
/mnt/defect39/815_Defect/紅框分類/20180327_done/紅框/crop/LS_R/TL57B1AA/G 4800TL7AR0DAX151100001-01.JPG
4800TL7AR0DAX151100001-01.JPG
currentstats 389 484 640 758
maxArea 4127
/mnt/defect39/815_Defect/紅框分類/20180327_done/紅框/crop/LS_R/TL57B1AA/G 4800TL7AR0DAX157100001-01.JPG
4800TL7AR0DAX157100001-01.JPG
currentstats 373 516 560 784
maxArea 3572
/mnt/defect39/815_Defect/紅框分類/20180327_done/紅框/crop/LS_R/TL57B1AA/G 4800TL7AR0DAX1B3100001-01.JPG
4800TL7AR0DAX1B3100001-01.JPG
currentstats 351 564 549 776
maxArea 3187
/mnt/defect39/815_Defect/紅框分類/20180327_done/紅框/crop/LS_R/TL57B1AA/G 4800TL7AR0DAX1L1100001-01.JPG
4800TL7AR0DAX1L1100001-01.JPG
currentstats 420 594 612 748
maxArea 2687
/mnt/defect39/815_Defect/紅框分類/20180327_done/紅框/crop/LS_R/TL57B1AA/G 4800TL7AR0DAX1NA100001-01.JPG
4800TL7AR0

currentstats 283 512 418 620
maxArea 1822
/mnt/defect39/815_Defect/紅框分類/20180327_done/紅框/crop/LS_R/TL57B1AA/G 4800TL7AR0KAQ132100001-01.JPG
4800TL7AR0KAQ132100001-01.JPG
currentstats 70 361 274 642
maxArea 3465
/mnt/defect39/815_Defect/紅框分類/20180327_done/紅框/crop/LS_R/TL57B1AA/G 4800TL7AR17AB116100001-01.JPG
4800TL7AR17AB116100001-01.JPG
currentstats 262 580 490 800
maxArea 3480
/mnt/defect39/815_Defect/紅框分類/20180327_done/紅框/crop/LS_R/TL57B1AA/G 4800TL7AR17AB1HA100001-01.JPG
4800TL7AR17AB1HA100001-01.JPG
currentstats 352 552 581 715
maxArea 3064
/mnt/defect39/815_Defect/紅框分類/20180327_done/紅框/crop/LS_R/TL57B1AA/G 4800TL7AR17AC1J7100001-01.JPG
4800TL7AR17AC1J7100001-01.JPG
currentstats 242 454 434 700
maxArea 3225
/mnt/defect39/815_Defect/紅框分類/20180327_done/紅框/crop/LS_R/TL57B1AA/G 4800TL7AR17AD119100001-01.JPG
4800TL7AR17AD119100001-01.JPG
currentstats 258 363 377 461
maxArea 1656
/mnt/defect39/815_Defect/紅框分類/20180327_done/紅框/crop/LS_R/TL57B1AA/G 4800TL7AR18AF115100001-01.JPG
4800TL7AR18

currentstats 344 547 478 778
maxArea 2831
/mnt/defect39/815_Defect/紅框分類/20180327_done/紅框/crop/PS_R/TL50B1AA/G 4800TL78P1SAA1SB100001-01.JPG
4800TL78P1SAA1SB100001-01.JPG
currentstats 200 880 342 1122
maxArea 2836
/mnt/defect39/815_Defect/紅框分類/20180327_done/紅框/crop/PS_R/TL50B1AA/G 4800TL78P1SAB131100001-01.JPG
4800TL78P1SAB131100001-01.JPG
currentstats 373 533 549 750
maxArea 3060
/mnt/defect39/815_Defect/紅框分類/20180327_done/紅框/crop/PS_R/TL50B1AA/G 4800TL78P1SAB1FC100001-01.JPG
4800TL78P1SAB1FC100001-01.JPG
currentstats 298 454 444 722
maxArea 3218
/mnt/defect39/815_Defect/紅框分類/20180327_done/紅框/crop/PS_R/TL50B1AA/G 4800TL78P1SAB1GC100001-01.JPG
4800TL78P1SAB1GC100001-01.JPG
currentstats 422 512 565 722
maxArea 2727
/mnt/defect39/815_Defect/紅框分類/20180327_done/紅框/crop/PS_R/TL50B1AA/G 4800TL78P1SAB1L2100001-01.JPG
4800TL78P1SAB1L2100001-01.JPG
currentstats 391 650 539 859
maxArea 2755
/mnt/defect39/815_Defect/紅框分類/20180327_done/紅框/crop/PS_R/TL50B1AA/G 4800TL78P1SAB1R6100001-01.JPG
4800TL78P

currentstats 476 587 574 722
maxArea 1705
/mnt/defect39/815_Defect/紅框分類/20180327_done/紅框/crop/PS_R/TL52D1AA/G 4800TL81T27AE1CD100001-01.JPG
4800TL81T27AE1CD100001-01.JPG
currentstats 490 605 589 750
maxArea 1792
/mnt/defect39/815_Defect/紅框分類/20180327_done/紅框/crop/PS_R/TL52D1AA/G 4800TL81T27AF17K100001-01.JPG
4800TL81T27AF17K100001-01.JPG
currentstats 574 432 684 571
maxArea 1857
/mnt/defect39/815_Defect/紅框分類/20180327_done/紅框/crop/PS_R/TL52D1AA/G 4800TL81T27AF1AL100001-01.JPG
4800TL81T27AF1AL100001-01.JPG
currentstats 265 601 359 690
maxArea 1371
/mnt/defect39/815_Defect/紅框分類/20180327_done/紅框/crop/PS_R/TL52D1AA/G 4800TL81T27AG109100001-01.JPG
4800TL81T27AG109100001-01.JPG
currentstats 253 452 359 548
maxArea 1482
/mnt/defect39/815_Defect/紅框分類/20180327_done/紅框/crop/PS_R/TL52D1AA/G 4800TL81T27AG10B100001-01.JPG
4800TL81T27AG10B100001-01.JPG
currentstats 494 629 584 699
maxArea 1125
/mnt/defect39/815_Defect/紅框分類/20180327_done/紅框/crop/PS_R/TL52D1AA/G 4800TL81T27AG18P100001-01.JPG
4800TL81T2

---------------------------------------------------------------------------------------

# use the crop image to find no rec and save with xml
#### xml is include x y xmax ymax

In [ ]:
import os,cv2
import numpy as np
import xml.etree.cElementTree as ET
import shutil


crop_image_path = "/home/kelly/Pictures/defect/0308_work/2800_DBC_0928-0305"
no_rec_path = "/home/kelly/Pictures/defect/0308_work/no_redRec"
xml_path = "/home/kelly/Pictures/defect/0308_work/xml"

no_rec_arr = os.listdir(no_rec_path)

#8800TL7BQ2NBC138100001-01

for root,dirs,files in os.walk(crop_image_path):
    for file in files:
        print(file)
        if file in no_rec_arr:
            print("!!")
            # create xml
            oriImg = cv2.imread(os.path.join(no_rec_path,file[:-4]+".jpg"))
            if not os.path.exists(os.path.join("/home/kelly/Pictures/defect/0308_work/data",root.split('/')[-1])):
                os.mkdir(os.path.join("/home/kelly/Pictures/defect/0308_work/data",root.split('/')[-1]))
            shutil.copy(os.path.join(no_rec_path,file[:-4]+".jpg"), os.path.join("/home/kelly/Pictures/defect/0308_work/data",root.split('/')[-1],file[:-4]+".jpg"))
            cropImg = cv2.imread(os.path.join(root,file[:-4]+".jpg"))
            
            result = cv2.matchTemplate(oriImg, cropImg, cv2.TM_CCOEFF_NORMED) 
            x_and_y = np.unravel_index(result.argmax(),result.shape)
            print(x_and_y)
            tree = ET.parse('/home/kelly/Pictures/fastrcnn/8800_0307/result/Annotations/8800AAHFJABXD1A2100001-01.xml')
            tree_root = tree.getroot()
            change_dict = {"filename":"8800TL76B0TAM1BA100001-01","width":oriImg.shape[1],"height":oriImg.shape[0],"xmin":x_and_y[1],"ymin":x_and_y[0],\
                                           "xmax":x_and_y[1]+cropImg.shape[1],"ymax":x_and_y[0]+cropImg.shape[0],"name":root.split('/')[-1]}

            for key in change_dict.keys():
                for e in tree_root.iter(key):
                    e.text = str(change_dict[key])
                    tree.write(os.path.join(xml_path,file[:-4]+".xml"))
        else:
            print("the file doesn't exist")
            

-----------------------------------------------------------------------------------------

## 將xml defect type 轉成 total

In [9]:
# all defect class to one class
# change xml object name to defect
import xml.etree.cElementTree as ET

source_xml_path = "/msl/home/kelly08385/Pictures/trained_defect/D300/Annotations"
des_xml_path = "/msl/home/kelly08385/Pictures/trained_defect/total_0629/Annotations"
source_img_path = "/msl/home/kelly08385/Pictures/trained_defect/D300/data"
des_img_path = "/msl/home/kelly08385/Pictures/trained_defect/total_0629/JPEGImages"
#des_img_path2 = "/msl/home/kelly08385/Pictures/trained_defect/total_0416/data/1156"

for root,dirs,files in os.walk(source_img_path):
    for file in files:
        dir = root.split('/')[-1]
        #mkdir(os.path.join(des_img_path2,dir))
        _source_xml_path = os.path.join(source_xml_path,file[:-4]+'.xml')
        if os.path.exists(_source_xml_path):
            tree = ET.parse(_source_xml_path)
            tree_root = tree.getroot()
            change_dict = {"name":'defect'}

            for key in change_dict.keys():
                for e in tree_root.iter(key):
                    e.text = str(change_dict[key])
                    tree.write(os.path.join(des_xml_path,file[:-4]+".xml"))
            shutil.copy(os.path.join(root,file),os.path.join(des_img_path,file))
            #shutil.copy(os.path.join(root,file),os.path.join(des_img_path2,dir,file))